<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [105]:
import pandas as pd
import psycopg2

In [107]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [108]:
query_3_1 = f'''
                select
                  count(*)
                from public.vacancies
            '''

In [109]:
display(pd.read_sql_query(query_3_1, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\4258740524.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_3_1, connection))


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [110]:
query_3_2 = f'''
select
  count(*)
from public.employers
'''

In [111]:
display(pd.read_sql_query(query_3_2, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\1161406050.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_3_2, connection))


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [112]:
query_3_3 = f'''
select
  count(*)
from public.areas
'''

In [113]:
display(pd.read_sql_query(query_3_3, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\1575138683.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_3_3, connection))


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [114]:
query_3_4 = f'''
select
  count(*) 
from public.industries
'''

In [115]:
display(pd.read_sql_query(query_3_4, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\3758540776.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_3_4, connection))


,count
0,294


Предварительный анализ данных показал, что у одного работадателя может быть несколько вакансий. Также один работадатель может представлять вакансии в разных сферах деятельности и в разных регионах.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [116]:
query_4_1 = f'''
select
  count(*) cnt,
  a.name area
from public.vacancies v
join public.areas a on v.area_id = a.id
group by a.id
order by 1 desc
'''

In [117]:
display(pd.read_sql_query(query_4_1, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\3356109972.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_4_1, connection))


,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы
...,...,...
764,1,Кизляр
765,1,Джизак
766,1,Эртиль
767,1,Арсеньев


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [118]:
query_4_2 = f'''
select
  count(*)
from public.vacancies v
where v.salary_from is not null OR v.salary_to is not null
'''

In [119]:
display(pd.read_sql_query(query_4_2, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\804385452.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_4_2, connection))


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [120]:
query_4_3 = f'''
select
  round(avg(v.salary_from)),
  round(avg(v.salary_to))
from public.vacancies v
'''

In [121]:
display(pd.read_sql_query(query_4_3, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\2685544595.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_4_3, connection))


,round,round
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [122]:
query_4_4 = f'''
select
  distinct v.schedule,
  v.employment,
  count(*)
from public.vacancies v
group by v.schedule, v.employment
order by 3 desc
'''

In [123]:
display(pd.read_sql_query(query_4_4, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\2351373252.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_4_4, connection))


,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [124]:
query_4_5 = f'''
select
v.experience,
  count(*)
from public.vacancies v
group by v.experience
order by 2
'''

In [125]:
display(pd.read_sql_query(query_4_5, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\2435147805.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_4_5, connection))


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

На основе детального анализа вакансий сделаны следующие предварительные выводы:
1. Ожидаемо, что количество вакансий по регионам распределены пропорционально размеру населения этих регионов.
2. Примерно только половина вакансий имеют заполненные верхний и нижний уровни вилки зарплат.
3. Средняя предполагаемая зарплата составляет от 70 до 110 тыс. рублей
4. Конечно, тип работы "полный день - полная занятость" до сих пор лидирует по количеству вакансий, но заметно, что тренд все больше и больше уходит в сторону удаленного формата работы.
5. Больше всего требуется на рынке соискателей,чей опыт составляет от 1 до 3 лет, меньше всего тех, у кого опыт более 6 лет, что ожидаемо, так как опытные специалисты ценятся больше и количество их должностей значительно меньше.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [126]:
query_5_1 = f'''
select
    e.name,
    count(*)
from public.vacancies v
join public.employers e on e.id = v.employer_id
group by e.id
order by 2 desc
limit 5
'''

In [127]:
display(pd.read_sql_query(query_5_1, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\3686457444.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_5_1, connection))


,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [128]:
query_5_2 = f'''
select
    a.name,
    count(distinct e.id) emp,
    count(v.id) vac
from public.areas a
left join public.vacancies v on a.id = v.area_id
left join public.employers e on e.area = a.id
where v.id is null
group by a.id
order by 2 desc
limit 1
'''

In [129]:
display(pd.read_sql_query(query_5_2, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\2141438708.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_5_2, connection))


,name,emp,vac
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [130]:
query_5_3 = f'''
select
    e.name,
    count(distinct v.area_id)
from public.employers e
left join public.vacancies v on v.employer_id = e.id
join public.areas a on e.area = a.id
group by e.id
order by 2 desc
limit 1
'''

In [131]:
display(pd.read_sql_query(query_5_3, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\3735917282.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_5_3, connection))


,name,count
0,Яндекс,181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [132]:
query_5_4 = f'''
select
    count(distinct e.id)
from public.employers e
left join public.employers_industries i on i.employer_id = e.id
where i.industry_id is null
'''

In [133]:
display(pd.read_sql_query(query_5_4, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\4279320806.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_5_4, connection))


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [134]:
query_5_5 = f'''
select
    e.name
from public.employers e
left join public.employers_industries i on i.employer_id = e.id
group by e.id
having count(distinct i.industry_id)=4
order by 1 asc
limit 1 offset 2
'''

In [135]:
display(pd.read_sql_query(query_5_5, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\2250716593.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_5_5, connection))


,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [136]:
query_5_6 = f'''
select
    count(distinct e.id)
from public.employers e
left join public.employers_industries i on i.employer_id = e.id
left join public.industries ind on ind.id = i.industry_id
where ind.name = 'Разработка программного обеспечения'
'''

In [137]:
display(pd.read_sql_query(query_5_6, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\1525759387.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_5_6, connection))


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [138]:
query_5_7 = f'''
with x as
(select
    a.name,
    count(*) cnt
from public.vacancies v
join public.employers e on e.id = v.employer_id
join public.areas a on v.area_id = a.id
where e.name = 'Яндекс'
group by a.id
having a.name = 'Москва' or a.name = 'Санкт-Петербург' or a.name = 'Новосибирск' or a.name = 'Екатеринбург' or a.name = 'Казань' or a.name = 'Красноярск' or a.name = 'Нижний Новгород' or a.name = 'Челябинск' or a.name = 'Уфа' or a.name = 'Самара' or a.name = 'Ростов-на-Дону' or a.name = 'Краснодар' or a.name = 'Омск' or a.name = 'Воронеж' or a.name = 'Пермь' or a.name = 'Волгоград'
)
select
    *
from x
UNION ALL
select
         'Total',
         SUM(x.cnt)
FROM x
'''

In [139]:
display(pd.read_sql_query(query_5_7, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\4223914822.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_5_7, connection))


,name,cnt
0,Москва,54.0
1,Санкт-Петербург,42.0
2,Екатеринбург,39.0
3,Новосибирск,35.0
4,Волгоград,24.0
5,Воронеж,32.0
6,Краснодар,30.0
7,Красноярск,23.0
8,Нижний Новгород,36.0
9,Омск,21.0


***

На основе детального анализа работодателей сделаны следующие предварительные выводы:
1. Ожидаемо, что по количеству вакансий ТОП-5 представлен компаниями "Яндекс", "Ростелеком", "СБЕР", "Тинькофф", "Газпром", они являются лидерами в сфере цифровой экономики; 
2. Есть ситуации, когда работадель в качестве своего региона выбирате "Россия", а вакансии выставляет непосредственно с уточнением региона;
3. Компания Яндекс публикует свои вакансии по всей России!;
4. 30% работадетелей предпочитают не указывать свою сферу деятельности;
5. Есть компании, у которых 4 сферы деятельности, одной из таких является компания 2ГИС;
6. Разработкой программного обеспечения занимаются более 10% компаний представленных на платформе.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [140]:
query_6_1 = f'''
select
    count(*)
from public.vacancies v
where (lower(v.name) like '%данн%') or (lower(v.name) like '%data%')
'''

In [141]:
display(pd.read_sql_query(query_6_1, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\3784264343.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_6_1, connection))


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [142]:
query_6_2 = f'''
select
    count(v.name)
from vacancies v
WHERE
(
    (name ILIKE '%data scientist%') OR (name ILIKE '%data science%') OR (name ILIKE '%исследователь данных%')
    OR ((name ILIKE '%ML%') AND (name NOT ILIKE '%HTML%'))
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%')
    AND (key_skills IS NOT NULL)
)
    AND (v.name ILIKE '%junior%'
    OR v.experience ILIKE '%Нет опыта%'
    OR v.employment ILIKE '%Стажировка%')
'''

In [143]:
display(pd.read_sql_query(query_6_2, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\799388316.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_6_2, connection))


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [144]:
query_6_3 = f'''
select
    count(v.name)
from public.vacancies v
WHERE
(
    (name ILIKE '%data scientist%') OR (name ILIKE '%data science%') OR (name ILIKE '%исследователь данных%')
    OR ((name ILIKE '%ML%') AND (name NOT ILIKE '%HTML%'))
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%')
)
AND
((key_skills ILIKE '%sql%')
OR (key_skills ILIKE '%postgres%'))
'''

In [145]:
display(pd.read_sql_query(query_6_3, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\621936193.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_6_3, connection))


,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [146]:
query_6_4 = f'''
select
    count(v.name)
from public.vacancies v
WHERE
(
    (name ILIKE '%data scientist%') OR (name ILIKE '%data science%') OR (name ILIKE '%исследователь данных%')
    OR ((name ILIKE '%ML%') AND (name NOT ILIKE '%HTML%'))
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%')
)
AND
key_skills ILIKE '%python%'
'''

In [147]:
display(pd.read_sql_query(query_6_4, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\1823800354.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_6_4, connection))


,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [148]:
query_6_5 = f'''
select
    (round(avg(length(key_skills) - length(replace(key_skills,CHR(9),''))+1),2))
from public.vacancies v
WHERE
(
    (v.name ILIKE '%data scientist%')
    OR (v.name ILIKE '%data science%')
    OR (v.name ILIKE '%исследователь данных%')
    OR (v.name  LIKE '%ML%' AND  (v.name NOT ILIKE '%html%'))
    OR (v.name ILIKE '%machine learning%')
    OR (v.name ILIKE '%машинн%обучен%')
    AND (key_skills IS NOT NULL)
)
'''

In [149]:
display(pd.read_sql_query(query_6_5, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\426166861.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_6_5, connection))


,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [150]:
query_6_6 = f'''
select
    v.experience,
    round((sum(coalesce(v.salary_from, v.salary_to))+ sum(coalesce(v.salary_to, v.salary_from)))/count(*)/2) avg
from public.vacancies v
WHERE
(
    (v.name ILIKE '%data scientist%')
    OR (v.name ILIKE '%data science%')
    OR (v.name ILIKE '%исследователь данных%')
    OR (v.name  LIKE '%ML%' AND  (v.name NOT ILIKE '%html%'))
    OR (v.name ILIKE '%machine learning%')
    OR (v.name ILIKE '%машинн%обучен%')
    AND (key_skills IS NOT NULL)
)
AND 
((v.salary_to IS NOT NULL) OR (v.salary_from IS NOT NULL))
group by v.experience
'''

In [151]:
display(pd.read_sql_query(query_6_6, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\113202061.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_6_6, connection))


,experience,avg
0,Нет опыта,74642.0
1,От 1 года до 3 лет,145235.0
2,От 3 до 6 лет,243114.0


***

1. Примерно 7% вакансий по рынку связаны с работой с данными;
2. Всего 51  вакансия из 533 на рынке DS подходит для начинающего специалиста, что меньше 10%;
3. В 229 вакансиях DS специалиста требуется знание SQL и postgres, что равняется 43%;
4. Знание python требуется в 357 вакансиях на позицию DS разработчика (вообще наверняка он требуется везде, просто это и так очевидно и многие даже не указывают);
5. В среднем работадатели указывают в требованиях 6-7 ключевых навыков;
6. Зарплата опытных DS разработчиков составляет в среднем более 240 тыс. рублей;

# Общий вывод по проекту

1. Исследование зарплатной вилки DS разработчиков и по базе в целом.

In [ ]:
query_1 = f'''
select
    round(avg(v.salary_from)) l,
    round(avg(v.salary_to)) h
from public.vacancies v
WHERE
(
    (v.name ILIKE '%data scientist%')
    OR (v.name ILIKE '%data science%')
    OR (v.name ILIKE '%исследователь данных%')
    OR (v.name  LIKE '%ML%' AND  (v.name NOT ILIKE '%html%'))
    OR (v.name ILIKE '%machine learning%')
    OR (v.name ILIKE '%машинн%обучен%')
    AND (key_skills IS NOT NULL)
)
AND 
((v.salary_to IS NOT NULL) OR (v.salary_from IS NOT NULL))
'''

In [ ]:
display(pd.read_sql_query(query_1, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\143658445.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_1, connection))


,l,h
0,147845.0,222373.0


Средняя зарплатная вилка по вакансиям, представленными в базе данных составляет от 71065.0	до 110537.0 руб., в то время как средняя зарплатная вилка на вакансии DS разработчиков, представленных в БД составляет от 147845.0	до 222373.0 руб., что примерно в два раза больше.

2. Исследования зависимости типа занятости и формата работы на среднюю зарплату.

In [ ]:
query_2 = f'''
select
  distinct v.schedule,
  v.employment,
  count(*),
  (avg(v.salary_to)+avg(v.salary_from))/2 average
from public.vacancies v
group by v.schedule, v.employment
having (avg(v.salary_to)+avg(v.salary_from))/2 is not null
order by 4 desc
'''

In [ ]:
display(pd.read_sql_query(query_2, connection))

C:\Users\borrn\AppData\Local\Temp\ipykernel_15252\3607577623.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(query_2, connection))


,schedule,employment,count,average
0,Гибкий график,Проектная работа,18,148836.500000
1,Удаленная работа,Полная занятость,7802,128982.809952
2,Гибкий график,Полная занятость,1593,125540.852090
3,Вахтовый метод,Полная занятость,367,123428.382292
4,Полный день,Проектная работа,141,94469.779240
5,Удаленная работа,Проектная работа,133,85873.507407
6,Полный день,Полная занятость,35367,85420.807238
7,Сменный график,Проектная работа,1,60000.000000
8,Сменный график,Стажировка,12,57125.000000
9,Полный день,Частичная занятость,347,54268.948603


В среденем вакансии на полную занятость оплачиваются больше, чем остальные (1 строка скорее исключение, так как выборка достаточно маленькая). Интересно то, что в среднем вакансии на удаленном формате работы при полной занятости оплачиваются больше, чем при полном дне и полной занятости (конечно, можно списать это на то, что удаленный формат в основном в IT сфере, где ЗП в целом больше). Это указывает на то, что тенденция по перемещению на удаленный формат работы специальностей, где это возможно сохранится. 

Так как ключевой целью проекта являлось создание модели машинного обучения, которая будет рекомендовать вакансии клиентам агентства, претендующим на позицию Data Scientist, то на основе проведенного анализа данных напрашиваются следующие выводы:

1. Однозначно, для рекомендации вакансий специалисту необходимо учитывать следующие факторы в большей степени влияющие на соответсвтие кандидата требованиям работадателя:
  а. Соответствие ключевых навыков специалиста запросам работадателей. Если прорабатывать эту фильтрацию более жестко, то может и не будет на одной вакансии 5000+ откликов людей, которые не соответствуют предъявляемым требованиям.
  б. Регион необходмо учитывать в тех случаях, когда не рассматривается удаленный формат работы. В случае с удаленным форматом можно выдавать все вакансии, которые допускают такой формат работы.
  в. Опыт работы. С учетом опыта работы, выбранного направления, ключевых навыков и региона можно автоматически определять кандитатам вилку зарплат на которую они могут рассчитывать(с возможностью личной корректировки, конечно), так рекомендовать вакансии будет гораздо проще на мой взгляд, так как данные будут взаимосвязаны (рекомендации зарплат будут формироваться на основе вакансий).

2. В целом вакансий, связанных с разработкой как таковой большинство.
3. Определенно, в крупных городах рынок труда гораздо больше развит в сравнении с остальными регионами (зависимость пропорциональна населению).
4. Тенденция по перемещению на удаленный формат работы специалистов будет сохраняться, так как количество вакансий большое, а в некоторых случаях оплата труда даже больше.
5. Очевидно, что с опытом работы зарплатные вилки увеличиваются, но в целом можно их четко разграничивать также на основании IT специальности.

На мой взгляд, рассматриваемые в проекте данные полностью соответсвуют целям и позволяют на их основании проводить дальнейшее исследование.